# Main CNN model for bat call classification

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
from keras import backend as K 
import cv2
import time
from sklearn.model_selection import train_test_split
import itertools_len as itertools
from itertools_len import product
import gc
from tensorflow.keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN, SMOTETomek
from tensorflow.keras import regularizers

2023-12-21 23:00:30.486399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 23:00:30.812571: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-21 23:00:30.812594: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-21 23:00:31.849150: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
# load image data s and reshape 
data = pd.read_pickle('./data/images_df_numerical.pkl')

# convert to numpy array
X, y = data['data'], data['Species']

In [5]:
# Alleiniges undersampling wird keinen Sinn machen, da wir extrem wenig Datenpunkte overall haben
# oversampling
smote = SMOTE()
X, y = np.stack(X), y.astype("int")
X_smote, y_smote = smote.fit_resample(X, y)
print("smote: ", pd.Series(y_smote).value_counts())
adasyn = ADASYN()
X_adasyn, y_adasyn = adasyn.fit_resample(X, y)
print("adasyn: ", pd.Series(y_adasyn).value_counts())
# Kombination aus over und undersampling
smoteenn = SMOTEENN()
X_smoteenn, y_smoteenn = smoteenn.fit_resample(X, y)
print("smoteenn: ", pd.Series(y_smoteenn).value_counts())
smotettomek = SMOTETomek()
X_smotettomek, y_smotettomek = smotettomek.fit_resample(X, y)
print("smotettomek: ", pd.Series(y_smotettomek).value_counts())

X_resampled = [X_smote, X_adasyn, X_smoteenn, X_smotettomek]
y_resampled = [y_smote, y_adasyn, y_smoteenn, y_smotettomek]

MemoryError: Unable to allocate 20.4 GiB for an array with shape (12624, 216432) and data type float64

In [ ]:
classes = y.unique()
image_size = X[0].size
samples = X.size
image_shape = (216,334,3) # height, width , channel
# reshape every row to the image, swap rgbs and scale to 0-1
X = np.array([
    cv2.cvtColor(row.reshape(image_shape), cv2.COLOR_BGR2RGB).astype('float32')/255. 
    for row in X])
y = np.array([row.astype('int32') for row in y])

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)

tf.keras.utils.set_random_seed(1)

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
tf.config.experimental.enable_op_determinism()

In [ ]:
number_of_classes = classes.size
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, min_delta=0.001, start_from_epoch=15, restore_best_weights=True)
epochs = 200
batch_size = 32
dropout_rate = 0.4 # https://www.kaggle.com/code/rafjaa/dealing-with-very-small-datasets interessant bzgl oberfitting
weight_decay_alpha = 0.01

def kaggle_model(optimizer):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=image_shape))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(weight_decay_alpha)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(number_of_classes, activation='softmax'))
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model

def create_optimizers(X_train) -> dict:
    s = 130 * len(X_train) // 32 # number of steps in 130 epochs (batch size = 32)
    exp_decay_sgd = ExponentialDecay(0.01, s, 0.1)
    exp_adam = ExponentialDecay(0.01, s, 0.95, staircase=True)

    momentum = 0.99
    sgd_exp = SGD(exp_decay_sgd, momentum=momentum)
    adam_exp = Adam(exp_adam)

    sgd = SGD(0.001, momentum=momentum)
    adam = Adam(0.001)

    return {"sgd_exp": sgd_exp, "adam_exp": adam_exp, "sgd": sgd, "adam": adam}

histories_with_params = list()

# Training and validating the model using KFold
for train_indezes, test_indezes in kfold.split(X, y):
    X_train, y_train = X[train_indezes], y[train_indezes]
    X_test, y_test = X[train_indezes], y[test_indezes]

    optimizers = create_optimizers(X_train)

    for optimizer_name, optimizer in optimizers.items():
        K.clear_session()
        model = kaggle_model(optimizer)
        history = model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            workers=1, # workers are number of cores
            callbacks=early_stopping,
            validation_split=0.2,
            verbose=1)
        model.save("cnn_files/model.keras", overwrite=True)
        history_with_param = {"optimizer": optimizer_name, "history": history}
        histories_with_params.append(history_with_param)
        del model
        K.clear_session()
        tf.reset_default_graph() 
        gc.collect()

number_of_epochs = len(history.history["accuracy"])
for history_with_param in histories_with_params:
    model = load_model(f"cnn_files/model_{history_with_param['optimizer']}.keras")
    test_score = round(model.evaluate(X_test, y_test)[1], 2)*100
    del model
    gc.collect()

    plt.figure()
    plt.plot(history_with_param["history"].history["accuracy"], label="train_data accuracy")
    plt.plot(history_with_param["history"].history["val_accuracy"], label="val_data accuracy")
    plt.scatter(number_of_epochs, test_score/100, label="test_data accuracy", marker="x", c="g")
    plt.title(f"opt: {history_with_param['optimizer']} Test Score: {test_score}%")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(loc="upper left")
    plt.savefig(f"./cnn_files/{history_with_param['optimizer']}.png",dpi=600)
    #plt.show()


In [ ]:
# weight pruning
import tensorflow_model_optimization as tfmot
validation_split = 0.1 # 10% of training set as validation

end_step = np.ceil(X.shape[0] / batch_size).astype(np.int32) * epochs

pruning_params = {
    # In this example, you start the model with 50% sparsity (50% zeros in weights) and end with 80% sparsity.
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                final_sparsity=0.80,
                                                                begin_step=0,
                                                                end_step=end_step)}
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

model_for_pruning = prune_low_magnitude(model, **pruning_params)
model_for_pruning.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history = model_for_pruning.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            workers=1, # workers are number of cores
            callbacks=[early_stopping, tfmot.sparsity.keras.UpdatePruningStep()],
            validation_split=0.2,
            verbose=1)